In [ ]:
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from tabulate import tabulate
from src.datasets import camvid
from src import evaluate
from src import plot
from src import predict
from src import predict_video
from src.tiramisu import tiramisu
from src.callbacks import PlotMetrics
from src.utils import history_to_results
%matplotlib inline

In [ ]:
# the location to save coarse training weights to
coarse_weights = 'models/Tiramisu103-CamVid11-coarse.h5'
# the location to save fine training weights to
fine_weights = 'models/Tiramisu103-CamVid11-fine.h5'

In [ ]:
# the size to reshape images to before transformation
target_size = (360, 480)
# the size to crop images to for coarse training
coarse_crop = (224, 224)
# the batch size during coarse training
coarse_batch = 3
# the size to crop images to for fine tune training
fine_crop = (352, 480)
# the batch size during fine training
fine_batch = 1

# Coarse Tuning

In [ ]:
camvid11 = camvid.CamVid(
    mapping=camvid.CamVid.load_mapping(),
    target_size=target_size,
    crop_size=coarse_crop,
    batch_size=coarse_batch,
    horizontal_flip=True,
    ignored_labels=['Void'],
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y)
# plot the images
_ = plot.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
model = tiramisu.tiramisu((*coarse_crop, 3), camvid11.n,
    class_weights=camvid11.class_mask,
    learning_rate=1e-3,
)
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_categorical_accuracy', patience=100),
    LearningRateScheduler(lambda _, lr: 0.995 * lr),
    ModelCheckpoint(coarse_weights, 
        monitor='val_categorical_accuracy', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='max'
    ),
    PlotMetrics(),
]

In [ ]:
# fit the model with the data. divide the steps per epoch by the batch size
history = model.fit_generator(generators['train'],
    epochs=200,
    steps_per_epoch=int(367 / coarse_batch),
    validation_data=generators['val'],
    validation_steps=101,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
history_to_results(history)

## Test Metrics

In [ ]:
model.load_weights(coarse_weights)

In [ ]:
metrics = evaluate.evaluate(model, generators['test'], 233,
    mask=camvid11.class_mask, 
    code_map=camvid11.discrete_to_label_map,
)
metrics.to_csv(coarse_weights + '.csv')
metrics

In [ ]:
print(tabulate(metrics, tablefmt='pipe', headers=('Metric', 'Value')))

## Qualitative Results

### Train

In [ ]:
X, y, p = predict.predict(model, generators['train'], camvid11)

In [ ]:
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
_ = plot.plot(X=X[1], y=y[1], y_pred=p[1], order=['X', 'y', 'y_pred'])

In [ ]:
_ = plot.plot(X=X[2], y=y[2], y_pred=p[2], order=['X', 'y', 'y_pred'])

### Validation

In [ ]:
X, y, p = predict.predict(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

### Test

In [ ]:
X, y, p = predict.predict(model, generators['test'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['test'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['test'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(model, generators['test'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

# Clear Session

remove the current model from memory.

In [ ]:
from keras import backend as K
K.clear_session()

# Fine Tuning

In [ ]:
camvid11 = camvid.CamVid(
    mapping=camvid.CamVid.load_mapping(),
    target_size=target_size,
    crop_size=fine_crop,
    batch_size=fine_batch,
    horizontal_flip=True,
    ignored_labels=['Void'],
)
generators = camvid11.generators()

In [ ]:
# get the next X, y training tuple
X, y = next(generators['train'])
# transform the onehot vector to an image
y = camvid11.unmap(y)
# plot the images
_ = plot.plot(X=X[0], y=y[0], order=['X', 'y'])

In [ ]:
# build the model for the image shape and number of labels
tune_model = tiramisu.tiramisu((*fine_crop, 3), camvid11.n,
    class_weights=camvid11.class_mask,
    learning_rate=1e-4,
)
# load the weights from the coarsely trained model
tune_model.load_weights(coarse_weights)
tune_model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_categorical_accuracy', patience=50),
    ModelCheckpoint(fine_weights, 
        monitor='val_categorical_accuracy', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='max'
    ),
    PlotMetrics(),
]

In [ ]:
# fit the model with the data.
history = tune_model.fit_generator(generators['train'],
    epochs=200,
    steps_per_epoch=int(367 / fine_batch),
    validation_data=generators['val'],
    validation_steps=101,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
history_to_results(history)

## Test Metrics

In [ ]:
tune_model.load_weights(fine_weights)

In [ ]:
metrics = evaluate.evaluate(tune_model, generators['test'], 233,
    mask=camvid11.class_mask, 
    code_map=camvid11.discrete_to_label_map,
)
metrics.to_csv(fine_weights + '.csv')
metrics

In [ ]:
print(tabulate(metrics, tablefmt='pipe', headers=('Metric', 'Value')))

## Qualitative Results

### Train

In [ ]:
X, y, p = predict.predict(tune_model, generators['train'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['train'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['train'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['train'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

### Validation

In [ ]:
X, y, p = predict.predict(tune_model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

In [ ]:
X, y, p = predict.predict(tune_model, generators['val'], camvid11)
_ = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])

### Test

In [ ]:
X, y, p = predict.predict(tune_model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])
fig.savefig('img/tiramisu/0.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p = predict.predict(tune_model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])
fig.savefig('img/tiramisu/1.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p = predict.predict(tune_model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])
fig.savefig('img/tiramisu/2.png', transparent=True, bbox_inches='tight')

In [ ]:
X, y, p = predict.predict(tune_model, generators['test'], camvid11)
fig = plot.plot(X=X[0], y=y[0], y_pred=p[0], order=['X', 'y', 'y_pred'])
fig.savefig('img/tiramisu/3.png', transparent=True, bbox_inches='tight')

# Video

In [ ]:
video_file = '0005VD.mp4'
video_path = camvid.videos.abs_path(video_file)
out_path = 'img/tiramisu/{}'.format(video_file)

In [ ]:
predict_video.predict_video(video_path, out_path, camvid11, tune_model, predict.predict)